In [ ]:
import pandas as pd
from model import LSTM_Tamas, train_lagged_forecaster, predict, plot_quantile_forecasts, LaggedPriceDataset_Tamas
from torch.utils.data import DataLoader
import torch

ModuleNotFoundError: No module named 'model'

In [ ]:
if __name__ == "__main__":
    data = pd.read_csv("formatted_data\formatted_data.csv")
    data["DA"] = data["DA"].fillna(0)

    lags = [24, 48, 72, 168]
    quantiles = [0.3, 0.5, 0.7]
    forecast_horizon = 24

    train_data = data.iloc[:-200]
    test_data = data.iloc[-200:]

    train_dataset = LaggedPriceDataset_Tamas(train_data, lags, forecast_horizon)
    test_dataset = LaggedPriceDataset_Tamas(test_data, lags, forecast_horizon)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    trained_model = train_lagged_forecaster(
        model_class=LSTM_Tamas,
        data=train_data,
        lags=lags,
        input_size=8,
        hidden_size=64,
        num_layers=2,
        dropout=0.1,
        forecast_horizon=forecast_horizon,
        attention_heads=4,
        quantiles=quantiles,
        num_epochs=10,
        batch_size=32,
        optimizer_class=torch.optim.Adam,
        learning_rate=0.001,
        device="cpu",
    )

    forecasts, true_values = predict(trained_model, test_loader, device="cpu")
    plot_quantile_forecasts(forecasts, true_values, quantiles=quantiles, forecast_horizon=forecast_horizon)